# 1) Criar o mecanismo de recomendação de produtos da Amazon, utilizando o algoritmo KNN

#Alunos:
1- Jônatas Luiz Gomes dos Santos - RA: 22353709
2- Pedro Augusto Amorim Barbosa Guedes - RA: 22303799

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


In [ ]:
# Carregar dados e renomear colunas
data = pd.read_csv('ratings_Electronics.csv', header=None)
data.columns = ["userId", "productId", "avaliação", "timestamp"]
data.head()

,userId,productId,avaliação,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1.365811e+09
1,A2CX7LUOHB2NDG,0321732944,5.0,1.341101e+09
2,A2NWSAGRHCP8N5,0439886341,1.0,1.367194e+09
3,A2WNBOD3WNDNKT,0439886341,3.0,1.374451e+09
4,A1GI0U4ZRJA8WN,0439886341,1.0,1.334707e+09


In [ ]:
#Retirando colunas inutéis para a solução
EstudoProdutos = data[["userId", "productId", "avaliação"]]
EstudoProdutos.head()

,userId,productId,avaliação
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0


In [ ]:
#Verificando se há valores nulos nas colunas
EstudoProdutos.isnull().sum()
EstudoProdutos.dropna(subset=["productId"], inplace=True)
#Jóia, tudo preenchido corretamente :)

<ipython-input-14-9f6dc6f26181>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EstudoProdutos.dropna(subset=["productId"], inplace=True)


In [ ]:
#Captando quantidade de avaliações por usuários
EstudoProdutos["userId"].value_counts()

#Com este resultado percebemos que determinados usuários avaliaram mais de um produto

,count
userId,
A5JLAU2ARJ0BO,357
A231WM2Z2JL0U3,249
A25HBO5V8S8SEA,162
AT6CZDCP4TRGA,99
AKT8TGIT6VVZ5,99
...,...
A3O7P3DGA0TG98,1
A3GRH6NEB9VLL,1
A2K08F17YEBUS4,1


In [ ]:
#Captando quantidade de avaliações por produto
QAvaliaçoesProdutos = EstudoProdutos["productId"].value_counts()
QAvaliaçoesProdutos.value_counts()


,count
count,
1,11248
2,5699
3,3712
4,2497
5,1887
...,...
442,1
443,1
446,1


In [ ]:
#Quanto mais avaliações um produto tem, maior é a assertividade que possuímos sobre sua qualidade.
#O ato de recomendar um produto especifico para um cliente pode ser um fator de grande impacto em uma loja
#Por isso iremos filtrar para apenas produtos com mil avaliações ou mais

EstudoProdutos1000Mais = QAvaliaçoesProdutos[QAvaliaçoesProdutos >= 1000].index
DefEstudoProdutos = EstudoProdutos[EstudoProdutos["productId"].isin(EstudoProdutos1000Mais)]
DefEstudoProdutos["productId"].value_counts()

#667 produtos estão aptos para recomendação

,count
productId,
B0002L5R78,9487
B0001FTVEK,5345
B00007E7JU,3523
B00007M1TZ,2608
B00004ZCJE,2547
B000652M6Y,2152
B00020S7XK,2140
B0007MXZB2,2080
B00001P4ZH,2075


In [ ]:
#Separando produtos pela quantidade de "nível" de avaliação
QuantProdsAval = DefEstudoProdutos["avaliação"].value_counts()
QuantProdsAval

,count
avaliação,
5.0,48653
4.0,15845
3.0,5877
1.0,5833
2.0,3799


In [ ]:
#Removendo avaliações feitas pelo mesmo usuário em um produto
DefEstudoProdutos = DefEstudoProdutos.drop_duplicates(subset=["userId", "productId"], inplace=False)


In [ ]:
DefEstudoProdutos.shape

(80007, 3)

In [ ]:
#Transformando id do produto em int, facilitando para o algoritmo
#DefEstudoProdutos["productId"] = DefEstudoProdutos["productId"].astype(int)

#Pelo ERRO apresentado, tem produtos que possuem letras em seu id então o tipo de varíavel é string mesmo

In [ ]:
#Criando um PIVOT para avaliar qual usuário avlaiou qual produto com a sua respectiva nota
Produtos_pivot = DefEstudoProdutos.groupby(['productId', 'userId']).mean()['avaliação'].unstack()

Produtos_pivot.head(20)

userId,A000681618A3WRMCK53V,A00328742CDZTXNDCB9XW,A0081833O8JMU2ODUD0V,A00988692Q9ZDJUD8BQSM,A00995931BE16NG4F52QC,A01262391CVDKK68IDCWS,A01293253GNI9N7MLGW7W,A014623426J5CM7M12MBW,A015639027WKW2102QO0S,A02058911H5ITPHXQ8YIX,...,AZZHG7U4YKOQH,AZZI4GIB6M3E6,AZZMDW27MUJR6,AZZQ4AHH7N9CH,AZZQFCVCFE8HL,AZZRXD27FMRE4,AZZV891DC7X6Q,AZZVLOF3WKLFW,AZZVYHVJ8BDUZ,AZZYTPFG6JF1K
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00001P4ZH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00001WRSJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00004SABB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00004SB92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00004T8R2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00004ZCJE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00005ATMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00005LEN4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Os valores que são nulos iremos preencher com ZERO
Produtos_pivot.fillna(0, inplace=True)
Produtos_pivot.head(20)

userId,A000681618A3WRMCK53V,A00328742CDZTXNDCB9XW,A0081833O8JMU2ODUD0V,A00988692Q9ZDJUD8BQSM,A00995931BE16NG4F52QC,A01262391CVDKK68IDCWS,A01293253GNI9N7MLGW7W,A014623426J5CM7M12MBW,A015639027WKW2102QO0S,A02058911H5ITPHXQ8YIX,...,AZZHG7U4YKOQH,AZZI4GIB6M3E6,AZZMDW27MUJR6,AZZQ4AHH7N9CH,AZZQFCVCFE8HL,AZZRXD27FMRE4,AZZV891DC7X6Q,AZZVLOF3WKLFW,AZZVYHVJ8BDUZ,AZZYTPFG6JF1K
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00001P4ZH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00001WRSJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004SABB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004SB92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004T8R2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004ZCJE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005ATMB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005LEN4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Importando csr_matrix e tranformando o data sete em uma matriz sparsa
from scipy.sparse import csr_matrix

Produtos_pivot_matriz = csr_matrix(Produtos_pivot)


In [ ]:
type(Produtos_pivot_matriz)

scipy.sparse._csr.csr_matrix

In [ ]:
#Criando e treinando o modelo preditivo
modelo = NearestNeighbors(algorithm= "brute")
modelo.fit(Produtos_pivot_matriz)

NearestNeighbors(algorithm='brute')

# 1.1) REALIZANDO PREVISÕES DE RECOMENDAÇÕES

In [ ]:
import numpy as np

# Inserir o produto
item_index = Produtos_pivot.index.get_loc("B000067O5G")

# Selecionando a linha
item_vector = Produtos_pivot_matriz[item_index]

# Transformando em (1, -1) para o KNN
item_vector = item_vector.reshape(1, -1)

# Fazendo sugestão de produtos
distances, sugestions = modelo.kneighbors(item_vector)

for i in range(len(sugestions)):
  print(Produtos_pivot.index[sugestions[i]])

Index(['B000067O5G', 'B00008SCFL', 'B000204SWE', 'B00022OBO2', 'B000095SB4'], dtype='object', name='productId')


In [ ]:
# Inserir o produto
item_index = Produtos_pivot.index.get_loc("B00007EDM8")

# Selecionando a linha
item_vector = Produtos_pivot_matriz[item_index]

# Transformando em (1, -1) para o KNN
item_vector = item_vector.reshape(1, -1)

# Fazendo sugestão de produtos
distances, sugestions = modelo.kneighbors(item_vector)

for i in range(len(sugestions)):
  print(Produtos_pivot.index[sugestions[i]])

Index(['B00007EDM8', 'B00008SCFL', 'B000204SWE', 'B00022OBO2', 'B000067O5G'], dtype='object', name='productId')


In [ ]:
# Inserir o produto
item_index = Produtos_pivot.index.get_loc("0972683275")

# Selecionando a linha
item_vector = Produtos_pivot_matriz[item_index]

# Transformando em (1, -1) para o KNN
item_vector = item_vector.reshape(1, -1)

# Fazendo sugestão de produtos
distances, sugestions = modelo.kneighbors(item_vector)

for i in range(len(sugestions)):
  print(Produtos_pivot.index[sugestions[i]])

Index(['0972683275', 'B00008SCFL', 'B000204SWE', 'B00022OBO2', 'B000067O5G'], dtype='object', name='productId')


In [ ]:
# Inserir o produto
item_index = Produtos_pivot.index.get_loc("B00004T8R2")

# Selecionando a linha
item_vector = Produtos_pivot_matriz[item_index]

# Transformando em (1, -1) para o KNN
item_vector = item_vector.reshape(1, -1)

# Fazendo sugestão de produtos
distances, sugestions = modelo.kneighbors(item_vector)

for i in range(len(sugestions)):
  print(Produtos_pivot.index[sugestions[i]])

Index(['B00004T8R2', 'B00008SCFL', 'B000204SWE', 'B000067O5G', 'B00022OBO2'], dtype='object', name='productId')


In [ ]:
# Inserir o produto
item_index = Produtos_pivot.index.get_loc("B00005T3G0")

# Selecionando a linha
item_vector = Produtos_pivot_matriz[item_index]

# Transformando em (1, -1) para o KNN
item_vector = item_vector.reshape(1, -1)

# Fazendo sugestão de produtos
distances, sugestions = modelo.kneighbors(item_vector)

for i in range(len(sugestions)):
  print(Produtos_pivot.index[sugestions[i]])

Index(['B00005T3G0', 'B00008SCFL', 'B000204SWE', 'B00022OBO2', 'B000067O5G'], dtype='object', name='productId')


# 2) Criar o mecanismo de recomendação de produtos da Amazon, utilizando o algoritmo K-MEANS

#Alunos:
1- Vitória de Almeida Verçosa - RA: 22103230
2- Ricardo de Carvalho Brito - RA:22107145
3-  - RA:

In [ ]:
#Improtando as blibiotecas usadas para o algoritimo K-Means e para normalização dos dados
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#Prepara os dados para serem usados no algoritimo de clusterização (K-Means)
scaler = StandardScaler()
produtos_scaled = scaler.fit_transform(Produtos_pivot)

In [ ]:
#Aplica o KMeans para realizar a clusterização dos dados normalizados
#Foi adicionada uma nova coluna ao PIVOT, chamada 'Cluster', onde cada linha representa o cluster correspondente ao produto
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
Produtos_pivot['Cluster'] = kmeans.fit_predict(produtos_scaled)

In [ ]:
#Exibe as 20 primeiras linhas da PIVOT após a clusterização dos dados
Produtos_pivot.head(20)

#A última coluna é referente aos clusters dos produtos

userId,A000681618A3WRMCK53V,A00328742CDZTXNDCB9XW,A0081833O8JMU2ODUD0V,A00988692Q9ZDJUD8BQSM,A00995931BE16NG4F52QC,A01262391CVDKK68IDCWS,A01293253GNI9N7MLGW7W,A014623426J5CM7M12MBW,A015639027WKW2102QO0S,A02058911H5ITPHXQ8YIX,...,AZZI4GIB6M3E6,AZZMDW27MUJR6,AZZQ4AHH7N9CH,AZZQFCVCFE8HL,AZZRXD27FMRE4,AZZV891DC7X6Q,AZZVLOF3WKLFW,AZZVYHVJ8BDUZ,AZZYTPFG6JF1K,Cluster
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00001P4ZH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
B00001WRSJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00004SABB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00004SB92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00004T8R2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00004ZCJE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
B00005ATMB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
B00005LEN4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
#Imprime os produtos agrupados em cada cluster após a aplicação do K-Means

def imprimir_clusters(df):
   for cluster in range(n_clusters):
    print(f"\nCluster {cluster}:")
    produtos_no_cluster = df[df['Cluster'] == cluster].index.tolist()
    for produtos in produtos_no_cluster:
          print(f" - {produtos}")

In [ ]:
#Exibe os clusters definidos por grupos e quais produtos pertencem a cada um deles
imprimir_clusters(Produtos_pivot)


Cluster 0:
 - B00007E7C8

Cluster 1:
 - 0972683275
 - B00001WRSJ
 - B00004SABB
 - B00004SB92
 - B00004T8R2
 - B00005ATMB
 - B00005LEN4
 - B00005N6KG
 - B00005T3G0
 - B000062VUO
 - B000065BPB
 - B000067O5G
 - B00006B7DA
 - B00006RVPW
 - B00007E7JU
 - B00007EDM8
 - B00007EDZG
 - B00007KDVI
 - B000089GN3
 - B00008SCFL
 - B000095SB4
 - B00009R6TA
 - B00009XVCZ
 - B0000BVYTV
 - B00017LSPI
 - B000204SWE
 - B00022OBO2
 - B0002CPBWI
 - B0002KVQBA
 - B0002L5R78
 - B0002SQ2P2
 - B0002Y5WZM
 - B000652M6Y
 - B0006B486K
 - B00077AA5Q
 - B0007LJHVA
 - B0007MXZB2
 - B0007XJSQC
 - B00081A2KY

Cluster 2:
 - B0001FTVEK

Cluster 3:
 - B00001P4ZH

Cluster 4:
 - B000065BP9

Cluster 5:
 - B00007M1TZ

Cluster 6:
 - B00004ZCJE

Cluster 7:
 - B00020S7XK


In [ ]:
#Função para recomendar produtos similares ao produto selecionado pelo usuário e informar a qual cluster cada produto pertence
def recomendar_produto(produto_input,df):

  #Encontrar o cluster do primeiro produto selecionado
  cluster_produto = df[df.index == produto_input]['Cluster'].values[0]

  #Filtrar os produtos que estão no mesmo cluster
  recomendacoes = df[df['Cluster'] == cluster_produto].index.tolist()
  recomendacoes.remove(produto_input)

  print(f"O Produto selecionado: '{produto_input}' pertence ao Cluster {cluster_produto}")
  print(f"Recomendacoes para quem gostou de '{produto_input}':")

  for produtos in recomendacoes:
    cluster_recomendado = df[df.index == produtos]['Cluster'].values[0]
    print(f" - {produtos} (Cluster {cluster_recomendado})")

#Em resumo, essa função pega um produto que o usuário gostou, encontra produtos semelhantes e os recomenda para o usuário

In [ ]:
#Testando a função de recomendação
#Exibe a qual cluster o produto escolhido pertence e lista quais outros produtos fazem parte desse mesmo cluster
Produto_teste = '0972683275'
recomendar_produto(Produto_teste, Produtos_pivot)

O Produto selecionado: '0972683275' pertence ao Cluster 1
Recomendacoes para quem gostou de '0972683275':
 - B00001WRSJ (Cluster 1)
 - B00004SABB (Cluster 1)
 - B00004SB92 (Cluster 1)
 - B00004T8R2 (Cluster 1)
 - B00005ATMB (Cluster 1)
 - B00005LEN4 (Cluster 1)
 - B00005N6KG (Cluster 1)
 - B00005T3G0 (Cluster 1)
 - B000062VUO (Cluster 1)
 - B000065BPB (Cluster 1)
 - B000067O5G (Cluster 1)
 - B00006B7DA (Cluster 1)
 - B00006RVPW (Cluster 1)
 - B00007E7JU (Cluster 1)
 - B00007EDM8 (Cluster 1)
 - B00007EDZG (Cluster 1)
 - B00007KDVI (Cluster 1)
 - B000089GN3 (Cluster 1)
 - B00008SCFL (Cluster 1)
 - B000095SB4 (Cluster 1)
 - B00009R6TA (Cluster 1)
 - B00009XVCZ (Cluster 1)
 - B0000BVYTV (Cluster 1)
 - B00017LSPI (Cluster 1)
 - B000204SWE (Cluster 1)
 - B00022OBO2 (Cluster 1)
 - B0002CPBWI (Cluster 1)
 - B0002KVQBA (Cluster 1)
 - B0002L5R78 (Cluster 1)
 - B0002SQ2P2 (Cluster 1)
 - B0002Y5WZM (Cluster 1)
 - B000652M6Y (Cluster 1)
 - B0006B486K (Cluster 1)
 - B00077AA5Q (Cluster 1)
 - B0007LJ

In [ ]:
#Função para calcular a distância euclidiana entre 2 vetores, para determinar a similaridade ou dissimilaridade entre os produtos
def distancia_euclidiana(vetor1, vetor2):
  return np.sqrt(np.sum((vetor1 - vetor2) ** 2))

In [ ]:
#Função para recomendar produtos aos usuários utilizando o método de agrupamento do K-Means e o cálculo da distância Euclidiana
#O objetivo é medir a distancia entre os produtos para encontrar os mais semelhantes
def recomendar_produtos_kmeans_euclidiana(produto_input,df,n_recomendacoes=5):

  #Encontrar o cluster do primeiro produto selecionado pelo usuário
  cluster_produto = df[df.index == produto_input]['Cluster'].values[0]

  #Buscar produtos do mesmo cluster(Grupo)
  produtos_no_cluster = df[df['Cluster'] == cluster_produto]

  #Remover o produto já escolhido pelo usuário do cluster para não recomendá-lo
  produtos_no_cluster = produtos_no_cluster.drop(produto_input, axis=0)

  #Lista para armazenar as distancias calculadas
  distancias = []

  #Vetor do produto de entrada(sem a coluna de cluster)
  vetor_produto_input = df.loc[produto_input].drop(['Cluster']).values

  #Compara o produto que você usuário gostou com os outros produtos do mesmo grupo e mede a "distância" entre eles. Quanto menor a distância, mais semelhantes os produtos são.
  for produtos in produtos_no_cluster.index:
    vetor_produto = produtos_no_cluster.loc[produtos].drop(['Cluster']).values
    distancia = distancia_euclidiana(vetor_produto_input, vetor_produto)
    distancias.append((produtos,distancia))

  #Depois de medir as distâncias, ela organiza os produtos pela similaridade, colocando os mais próximos primeiro.
  distancias_ordenadas = sorted(distancias, key=lambda x: x[1])

  #Exibe o nome do produto escolhido e as recomendações de produtos semelhantes
  print(f"O Produto selecionado: '{produto_input}' pertence ao Cluster {cluster_produto}")
  print(f"Recomendacoes para quem gostou de '{produto_input}'")

  for i in range(n_recomendacoes):
    produto_recomendado, dist = distancias_ordenadas[i]
    print(f" - {produto_recomendado} (Distancia euclidiana: {dist:2f})")

In [ ]:
#Testar a função de recomendação que foi definida anteriormente

Produto_teste = '0972683275' #Exemplo de Produto
recomendar_produtos_kmeans_euclidiana(Produto_teste, Produtos_pivot)

O Produto selecionado: '0972683275' pertence ao Cluster 1
Recomendacoes para quem gostou de '0972683275'
 - B00008SCFL (Distancia euclidiana: 192.348642)
 - B000204SWE (Distancia euclidiana: 192.826347)
 - B000067O5G (Distancia euclidiana: 194.697201)
 - B00022OBO2 (Distancia euclidiana: 194.697201)
 - B000095SB4 (Distancia euclidiana: 195.504476)


In [ ]:
#Essa função recomenda produtos similares ao primeiro produto escolhido pelo usuário e informa a qual cluster cada produto pertence
def recomendar_produto(produto_input,df):
  #Encontra o cluster do primeiro produto selecionado
  cluster_produto = df[df.index == produto_input]['Cluster'].values[0]

  #Filtra os produtos do mesmo cluster
  recomendacoes = df[df['Cluster'] == cluster_produto].index.tolist()
  recomendacoes.remove(produto_input)

  print(f"O produto selecionado: '{produto_input}' pertence ao Cluster {cluster_produto}")
  print(f"Recomendacoes para quem gostou de '{produto_input}':")

  for produtos in recomendacoes:
    cluster_recomendado = df[df.index == produtos]['Cluster'].values[0]
    print(f" - {produtos} (Cluster {cluster_recomendado})")

In [ ]:
#Verificação do uso da RAM
import psutil

ram = psutil.virtual_memory()
print(f"Uso de RAM: {ram.percent}% de {ram.total / (1024**3):.2f} GB")

Uso de RAM: 14.3% de 12.67 GB


# Conclusão

In [ ]:
#
Escolha o KNN para classificação supervisionada (com rótulos)
e o K-Means para segmentação exploratória (sem rótulos) no banco de dados de produtos.